In [1]:
"""
Extract attributes for FRP observations

Data sources:
- LANDFIRE Canopy Bulk Density (CBD)
- LANDFIRE Canopy Base Height (CBH)
- LANDFIRE Tree Canopy Cover (TCC)

maxwell.cook@colorado.edu

"""

import os, time
import numpy as np

import pandas as pd
import geopandas as gpd
import rioxarray as rxr
import rasterio as rio
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from rasterstats import zonal_stats

proj = 'EPSG:5070'

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'

print("Complete")

Complete


In [2]:
# Functions

# Calculate zonal statistics
def calc_zonal_stats(vectors, raster_path, attr, stats=['mean'], categorical=False, geojson_out=True):
    begin = time.time()    

    # Calculate the zonal mean
    zs = zonal_stats(vectors=vectors[['VID','geometry']], raster=raster_path, stats=stats, categorical=categorical, geojson_out=geojson_out)
    
    # Convert the list of dicts to a DataFrame, extract properties
    zs_df = pd.DataFrame(zs)

    # Tidy the columns
    zs_df['VID'] = zs_df['properties'].apply(lambda x: x.get('VID'))
    zs_df['mean'] = zs_df['properties'].apply(lambda x: x.get('mean'))
    zs_df = zs_df[['VID','mean']]
    zs_df.rename(columns={'mean':f'{attr}_mean'}, inplace=True)

    print(zs_df.head())

    print(f"Total elapsed time for the current grid: {round((time.time() - begin))} seconds.")

    del zs
    
    return pd.DataFrame(zs_df)


## Extract LANDFIRE attributes for VIIRS observations

We will calculate zonal statistics for LANDFIRE Canopy Bulk Density (CBD), Canopy Base Height (CBH), and tree canopy cover (TCC).

In [3]:
# Load the VIIRS 375m observations
frp_plots = gpd.read_file(os.path.join(maindir,'aspen-fire/Aim2/data/spatial/mod/VIIRS/viirs_plots_fired_events_west_aspen.gpkg'))
print(frp_plots.head())

      VID   LATITUDE   LONGITUDE   ACQ_DATE ACQ_TIME CONFIDENCE    FRP  \
0  876446  37.507347 -112.675858 2020-11-18     1954          n  24.28   
1  876447  37.502945 -112.674408 2020-11-18     1954          n  24.28   
2  876448  37.504166 -112.669052 2020-11-18     1954          n  24.28   
3  876763  37.512333 -112.681885 2020-11-19     1936          n   7.04   
4  876764  37.513638 -112.677109 2020-11-19     1936          n   7.04   

  DAYNIGHT fired_id              ig_date  ... mn_grw_km2  mu_grw_km2  \
0        D    44927  2020-11-16 00:00:00  ...   0.214659    4.055308   
1        D    44927  2020-11-16 00:00:00  ...   0.214659    4.055308   
2        D    44927  2020-11-16 00:00:00  ...   0.214659    4.055308   
3        D    44927  2020-11-16 00:00:00  ...   0.214659    4.055308   
4        D    44927  2020-11-16 00:00:00  ...   0.214659    4.055308   

            mx_grw_dte      ig_utm_x      ig_utm_y     tot_perim  pct_aspen  \
0  2020-11-17 00:00:00  4.167730e+06 -9.943

In [23]:
start = time.time()    

# Read in the LANDFIRE layers
cbd_path = os.path.join(maindir,'data/landcover/LANDFIRE/LF2020_CBD_200_CONUS/Tif/LC20_CBD_200.tif')
cbh_path = os.path.join(maindir,'data/landcover/LANDFIRE/LF2020_CBH_200_CONUS/Tif/LC20_CBH_200.tif')
tcc_path = os.path.join(maindir,'data/landcover/LANDFIRE/LF2020_CC_200_CONUS/Tif/LC20_CC_200.tif')

lf_paths = [cbd_path, cbh_path, tcc_path]

results = []
for lf in lf_paths:
    print(f'Processing {os.path.basename(lf)}')
    name = os.path.basename(lf).split('_')[1]
    
    # Calculate the zonal statistics
    df = calc_zonal_stats(vectors=frp_plots[['VID','geometry']], raster_path=lf, attr=name)
    
    results.append(df)

print(f"Total elapsed time: {round((time.time() - start)/60)} minutes")

Processing LC20_CBD_200.tif
      VID  CBD_mean                                           geometry
0  876446    3.1808  {'type': 'Polygon', 'coordinates': (((-1452319...
1  876447    5.1168  {'type': 'Polygon', 'coordinates': (((-1452281...
2  876448    5.6992  {'type': 'Polygon', 'coordinates': (((-1451795...
3  876763    5.0400  {'type': 'Polygon', 'coordinates': (((-1452741...
4  876764    3.4704  {'type': 'Polygon', 'coordinates': (((-1452304...
Total elapsed time for the current grid: 392 seconds.
Processing LC20_CBH_200.tif
      VID  CBH_mean                                           geometry
0  876446    8.9424  {'type': 'Polygon', 'coordinates': (((-1452319...
1  876447    9.8512  {'type': 'Polygon', 'coordinates': (((-1452281...
2  876448    8.7392  {'type': 'Polygon', 'coordinates': (((-1451795...
3  876763    3.7440  {'type': 'Polygon', 'coordinates': (((-1452741...
4  876764    3.8160  {'type': 'Polygon', 'coordinates': (((-1452304...
Total elapsed time for the current gri

In [31]:
# Merge the results and join back to the FRP data
df1 = pd.merge(results[0][['VID','CBD_mean']], results[1][['VID','CBH_mean']], on='VID', how='inner')
df = pd.merge(df1, results[2][['VID','CC_mean']], on='VID', how='inner')

del df1

print(df.head())

# Merge back to the original FRP spatial data

frp_gdf = pd.merge(frp_plots, df, on='VID', how='inner')
print(len(frp_gdf['VID'].unique()))

# Save the spatial data
frp_gdf.to_file(os.path.join(maindir,'aspen-fire/Aim2/data/spatial/mod/VIIRS/viirs_plots_fired_events_west_aspen_LF.gpkg'))

      VID  CBD_mean  CBH_mean  CC_mean
0  876446    3.1808    8.9424   18.680
1  876447    5.1168    9.8512   27.384
2  876448    5.6992    8.7392   31.528
3  876763    5.0400    3.7440   23.864
4  876764    3.4704    3.8160   17.560
65124
